<a href="https://colab.research.google.com/github/ui2030/Test_A/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from glob import glob

import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import seaborn as sns

from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import datasets
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization



def save_history(history, model_name):
    hist_df = pd.DataFrame(history.history)

    # save to json:
    hist_json_file = model_name + '_history.json'
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)

    # or save to csv:
    hist_csv_file = model_name + '_history.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)


def plot_accuracy_from_history(history, history_file_name):
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    epochs = range(len(acc))

    sns.lineplot(x=epochs, y=acc, label='Training Accuracy')
    sns.lineplot(x=epochs, y=val_acc, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.savefig(history_file_name + '_acc.png')
    plt.figure()


def plot_loss_from_history(history, history_file_name):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))

    sns.lineplot(x=epochs, y=loss, label='Training Loss')
    sns.lineplot(x=epochs, y=val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig(history_file_name + '_loss.png')
    plt.figure()


def do_history_stuff(history, history_file_name):
    save_history(history, history_file_name)
    plot_accuracy_from_history(history, history_file_name)
    plot_loss_from_history(history, history_file_name)



gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import warnings
warnings.filterwarnings(action='ignore')

path = '/content/drive/MyDrive/Colab Notebooks/final_symbols_split_ttv/'

training_images = []
training_labels = []

for filename in glob(path + "*"):
    for img in glob(filename + "/*.png"):
        an_img = PIL.Image.open(img).convert('RGB') # read img
        an_img = an_img.resize((32, 32))
        img_array = np.array(an_img)  # img to array
        training_images.append(img_array)  # append array to training_images
        label = filename.split('\\')[-1]  # get label
        training_labels.append(label)  # append label


training_images = np.array(training_images)
training_labels = np.array(training_labels)

le = LabelEncoder()
training_labels = le.fit_transform(training_labels)
training_labels = training_labels.reshape(-1, 1)

print(training_images.shape)
print(training_labels.shape)

path = '/content/drive/MyDrive/MLDL/cacpj/dataset/test/'

test_images = []
test_idx = []

flist = sorted(glob(path + '*.png'))

for filename in flist:
    an_img = PIL.Image.open(filename).convert('RGB')  # read333333333333333 img
    an_img = an_img.resize((32, 32))
    img_array = np.array(an_img)  # img to array
    test_images.append(img_array)  # append array to training_images
    label = filename.split('\\')[-1]  # get id
    test_idx.append(label)  # append id

test_images = np.array(test_images)

print(test_images.shape)
print(test_idx[0:5])

tf.random.set_seed(42)

# image_generator = ImageDataGenerator(
#     shear_range=0.2,
#     zoom_range=0.1,
#     width_shift_range=0.05,
#     height_shift_range=0.05,
#     cval=255,
#     fill_mode='constant'
# )

# training_image_aug = image_generator.flow(training_images, np.zeros(42749), batch_size=42749, shuffle=False, seed = 42).next()[0]

# training_images = np.concatenate((training_images,
#                                   training_image_aug))

# training_labels = np.concatenate((training_labels,
#                                   training_labels))



training_labels = tf.one_hot(training_labels, 14) #one-hot 기법 적용
training_labels = np.array(training_labels)
training_labels = training_labels.reshape(-1,14) #one-hot 기법을 적용했다면, shape을 바꿔줍니다.

# print(training_images.shape)
# print(training_labels.shape)

X_train, X_valid, y_train, y_valid = train_test_split(training_images,
                                                      training_labels,
                                                      test_size=0.1,
                                                      stratify = training_labels,
                                                      random_state=42,
                                                      shuffle = True)

X_test = test_images

print('X_train 크기:',X_train.shape)
print('y_train 크기:',y_train.shape)
print('X_valid 크기:',X_valid.shape)
print('y_valid 크기:',y_valid.shape)
print('X_test  크기:',X_test.shape)

X_train = X_train / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0



base_model = VGG16(include_top=False, pooling = 'avg' , input_shape = (32, 32, 3), weights = "imagenet")

for layer in base_model.layers:
    layer.trainable = False


vgg_x = Flatten()(base_model.output)
vgg_x = Dense(64, activation= 'relu')(vgg_x)
vgg_x = Dense(14, activation = 'softmax')(vgg_x)
model_vgg = tf.keras.Model(base_model.input, vgg_x)

model_vgg.summary()

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_vgg.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.01),
                  loss = 'categorical_crossentropy',
                  metrics=['acc'])

vgg16_history = model_vgg.fit(X_train, y_train, epochs = 1, validation_data=(X_valid, y_valid)
                                 ,batch_size= 8)

for layer in base_model.layers[15:]:
    layer.trainable = True


model_vgg.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
                  loss = 'categorical_crossentropy',
                  metrics=['acc'])

vgg16_history = model_vgg.fit(X_train, y_train, epochs = 100, validation_data=(X_valid, y_valid)
                                 ,batch_size= 16)


do_history_stuff(vgg16_history, 'vgg16_model')


1 Physical GPUs, 1 Logical GPUs
(0,)
(0, 1)
(0,)
[]


ValueError: ignored